# HW 1

In [1]:
import pandas as pd

In [2]:
# Read the Parquet file into a DataFrame
df = pd.read_parquet('./data/yellow_tripdata_2022-01.parquet')

df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0


## Q1

In [3]:
#There are 19 columns
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2463931 entries, 0 to 2463930
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[ns]
 2   tpep_dropoff_datetime  datetime64[ns]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

## Q2

In [4]:
# Calculate the duration and create the new column
df['duration'] = df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']
df['duration'] = df.duration.dt.total_seconds() / 60
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0,17.816667
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0,8.400000
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0,8.966667
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0,10.033333
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0,37.533333


In [5]:
#The standard deviation for trip duration was 45 minutes and 26 seconds (or 46.45 minutes)
df['duration'].std()

46.44530513776499

## Q3

In [6]:
#Create a new dataframe without outliers, between 1 and 60 minute trip durations
len1 = df.shape[0]
df = df[(df['duration'] >= 1) & (df['duration'] <= 60)].copy()

#Roughly 98% of the records remain
df.shape[0]/len1

0.9827547930522406

## Q4

In [7]:
from sklearn.feature_extraction import DictVectorizer

# Select the pickup and dropoff location ID columns
selected_columns = ['PULocationID', 'DOLocationID']
df[selected_columns] = df[selected_columns].fillna(-1).astype('int')
df[selected_columns] = df[selected_columns].astype('str')

# Convert DataFrame to a list of dictionaries
dict_list = df[selected_columns].to_dict(orient='records')

# Initialize the dictionary vectorizer
vectorizer = DictVectorizer()

# Fit the dictionary vectorizer on the list of dictionaries
feature_matrix = vectorizer.fit_transform(dict_list)

# There are 515 columns
feature_matrix.shape

(2421440, 515)

## Q5

In [8]:
from sklearn.linear_model import LinearRegression

# Prepare the target variable (e.g., duration)
target_variable = df['duration'].values

# Initialize the linear regression model
model = LinearRegression()

# Train the linear regression model
model.fit(feature_matrix, target_variable)

LinearRegression()

In [9]:
import numpy as np
from sklearn.metrics import mean_squared_error
# Make predictions using the trained model
predictions = model.predict(feature_matrix)

# Calculate the RMSE
rmse = np.sqrt(mean_squared_error(target_variable, predictions))

# RMSE of 6.986
print("RMSE:", rmse)

RMSE: 6.986190836477672


## Q6

In [11]:
# Read the Parquet file into a DataFrame
val = pd.read_parquet('./data/yellow_tripdata_2022-02.parquet')
val['duration'] = val['tpep_dropoff_datetime'] - val['tpep_pickup_datetime']
val['duration'] = val.duration.dt.total_seconds() / 60
val = val[(val['duration'] >= 1) & (val['duration'] <= 60)].copy()

val.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,1,2022-02-01 00:06:58,2022-02-01 00:19:24,1.0,5.40,1.0,N,138,252,1,17.0,1.75,0.5,3.90,0.00,0.3,23.45,0.0,1.25,12.433333
1,1,2022-02-01 00:38:22,2022-02-01 00:55:55,1.0,6.40,1.0,N,138,41,2,21.0,1.75,0.5,0.00,6.55,0.3,30.10,0.0,1.25,17.550000
2,1,2022-02-01 00:03:20,2022-02-01 00:26:59,1.0,12.50,1.0,N,138,200,2,35.5,1.75,0.5,0.00,6.55,0.3,44.60,0.0,1.25,23.650000
3,2,2022-02-01 00:08:00,2022-02-01 00:28:05,1.0,9.88,1.0,N,239,200,2,28.0,0.50,0.5,0.00,3.00,0.3,34.80,2.5,0.00,20.083333
4,2,2022-02-01 00:06:48,2022-02-01 00:33:07,1.0,12.16,1.0,N,138,125,1,35.5,0.50,0.5,8.11,0.00,0.3,48.66,2.5,1.25,26.316667


In [13]:
# Select the pickup and dropoff location ID columns
selected_columns = ['PULocationID', 'DOLocationID']
val[selected_columns] = val[selected_columns].fillna(-1).astype('int')
val[selected_columns] = val[selected_columns].astype('str')

# Convert DataFrame to a list of dictionaries
val_dict_list = val[selected_columns].to_dict(orient='records')

# Fit the dictionary vectorizer on the list of dictionaries
val_matrix = vectorizer.transform(val_dict_list)

# Prepare the target variable (e.g., duration)
val_target_variable = val['duration'].values

# Make predictions using the trained model
val_predictions = model.predict(val_matrix)

# Calculate the RMSE
rmse = np.sqrt(mean_squared_error(val_target_variable, val_predictions))

# RMSE of 7.786
print("RMSE:", rmse)

RMSE: 7.78640879016696
